<a href="https://colab.research.google.com/github/ricefan-tech/Transformer/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
import pdb


# Transformer skeleton

In [2]:
class MyEmbedding(nn.Module):
  def __init__(self, d_model, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, input):
    return self.embedding(input)

class MyPositionalEncoding(nn.Module):
  def __init__(self, max_seq_len, d_model):
    super().__init__()
    self.positionalEncoding = nn.Embedding(max_seq_len, d_model)

  def forward(self, input):
    # embeddings look up the positions of the input entries and give the d_model-dimensional embedding
    # pdb.set_trace()
    seq_len = input.size(1)
    positional_encoding = self.positionalEncoding(torch.arange(seq_len, device=input.device)).unsqueeze(0) # need 2 dimension for broadcasting, and size(1) is seq
    return positional_encoding

class MyMultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super().__init__()
    self.Q_proj = nn.Linear(d_model, d_model, bias=False)
    self.K_proj = nn.Linear(d_model, d_model, bias=False)
    self.V_proj = nn.Linear(d_model, d_model, bias=False)
    self.num_heads = num_heads

  def forward(self, q_input, v_input, k_input, padding_mask=None, causal_mask=None):

    Q = self.Q_proj(q_input) # now has shape (batch_size, seq_len, d_model)
    K = self.K_proj(k_input)
    V = self.V_proj(v_input)

    # before attn score calculation need to reshape into multi head
    batch_size, seq_len, d_model = Q.size()
    Q = Q.reshape(batch_size, seq_len, self.num_heads, d_model//self.num_heads).transpose(1, 2) #d_model must be divisible by num_heads
    K = K.reshape(batch_size, seq_len, self.num_heads, d_model//self.num_heads).transpose(1, 2)
    V = V.reshape(batch_size, seq_len, self.num_heads, d_model//self.num_heads).transpose(1, 2)

    if padding_mask is not None:
      # padding mask is of shape (batch_size, seq_len), needs to be broadcasted to match Q@K.T which is (batch_size, num_head, seq_len, seq_len)
      padding_mask = padding_mask.unsqueeze(1).unsqueeze(2)
      padding_mask = padding_mask==0
      attention_scores = Q @ K.transpose(-2, -1) / math.sqrt(d_model//self.num_heads)
      attention_scores = attention_scores.masked_fill(padding_mask, float("-inf"))

    else:
      attention_scors = Q @ K.transpose(-2, -1) / math.sqrt(d_model//self.num_heads)

    # pdb.set_trace()
    if causal_mask is not None:
      # causal_mask is lower triangular matrix of shape target_seq_len, target_seq_len
      causal_mask = causal_mask.unsqueeze(0).unsqueeze(1).to(input.device)
      attention_scores = attention_scores.masked_fill(causal_mask, float("-inf"))

    attention_weights = F.softmax(attention_scores, dim=-1) @ V # is shape (batch_size, num_head, seq_len, head_dim)
    return attention_weights.reshape(batch_size, seq_len, d_model)


class MyFeedForwardNetwork(nn.Module):
  def __init__(self, d_model, ff_hidden):
    super().__init__()
    self.layer1 = nn.Linear(d_model, ff_hidden)
    self.layer2 = nn.Linear(ff_hidden, d_model)
    self.relu = nn.ReLU()

  def forward(self, input):
    return self.layer2(self.relu(self.layer1(input)))


class MyLayerNorm(nn.Module):
  def __init__(self, d_model, eps=1e-6):
    super().__init__()
    self.eps = eps #stabuliser for division

    self.gamma = nn.Parameter(torch.ones(d_model))
    self.beta = nn.Parameter(torch.zeros(d_model))

  def forward(self, input):
    mean = input.mean(dim=-1, keepdim=True)
    var = input.var(dim=-1, keepdim=True) #keeps broadcasted shape
    # pdb.set_trace()
    normalised_input = (input - mean)/ torch.sqrt( var+ self.eps)
    return self.gamma * normalised_input + self.beta


class MyEncoderLayer(nn.Module):
  def __init__(self, vocab_size, d_model, num_heads, ff_hidden):
    super().__init__()
    self.multiheadattention = MyMultiHeadAttention(d_model, num_heads)
    self.layer_norm = MyLayerNorm(d_model)
    self.layer_norm2 = MyLayerNorm(d_model)
    self.ff_network = MyFeedForwardNetwork(d_model, ff_hidden)

  def forward(self, input, padding_mask):
    attention_weights = self.multiheadattention(input, input, input, padding_mask=padding_mask, causal_mask=None)
    resid_conn = input+attention_weights
    noramlised_resid_conn = self.layer_norm(resid_conn)
    ff_network = self.ff_network(noramlised_resid_conn)
    resid_conn2 = noramlised_resid_conn+ff_network
    normalised_resid_conn = self.layer_norm2(resid_conn2)
    return normalised_resid_conn


class MyEncoder(nn.Module):
  def __init__(self, vocab_size, max_seq_len, d_model, num_heads, ff_hidden, num_encoder_layers):
    super().__init__()
    self.embedding_layer = MyEmbedding(d_model, vocab_size)
    self.positional_encoding = MyPositionalEncoding(max_seq_len, d_model)
    self.encoders_layers = nn.ModuleList([MyEncoderLayer(vocab_size, d_model, num_heads, ff_hidden) for _ in range(num_encoder_layers)])

  def forward(self, input, padding_mask):
    input = input.long()
    embed_input = self.embedding_layer(input)
    input = embed_input + self.positional_encoding(input)
    for layer in self.encoders_layers:
      input = layer(input, padding_mask)
    return input


class MyOutputLayer(nn.Module):
  def __init__(self, d_model, vocab_size):
    super().__init__()
    self.W_output = nn.Linear(d_model, vocab_size)

  def forward(self, input):
    return self.W_output(input)

class MyDecoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, ff_hidden):
    super().__init__()

    self.multiheadattention = MyMultiHeadAttention(d_model, num_heads)
    self.layer_norm = MyLayerNorm(d_model)
    self.layer_norm2 = MyLayerNorm(d_model)
    self.layer_norm3 = MyLayerNorm(d_model)
    self.layer_norm4 = MyLayerNorm(d_model)
    self.ff_network = MyFeedForwardNetwork(d_model, ff_hidden)
    self.ff_network2 = MyFeedForwardNetwork(d_model, ff_hidden)
    self.cross_multiheadattention = MyMultiHeadAttention(d_model, num_heads)

  def forward(self, input, encoder_output, padding_mask):
    # self attention first
    seq_len = input.size(1) # input is batch_size, seq_len
    causal_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)
    # ocnvert to boolean for the masked_fill() function
    causal_mask = causal_mask == 0 # masked_Fill fills the positions with -inf that are True
    attention_weights = self.multiheadattention(input, input, input, padding_mask=padding_mask, causal_mask=causal_mask)
    resid_conn = input+attention_weights
    noramlised_resid_conn = self.layer_norm(resid_conn)
    ff_network = self.ff_network(noramlised_resid_conn)
    resid_conn2 = noramlised_resid_conn+ff_network
    normalised_resid_conn = self.layer_norm2(resid_conn2)

    crossattention_weights = self.cross_multiheadattention(normalised_resid_conn, encoder_output, encoder_output, padding_mask=padding_mask)
    resid_conn = normalised_resid_conn+crossattention_weights
    noramlised_resid_conn = self.layer_norm3(resid_conn)
    ff_network = self.ff_network2(noramlised_resid_conn)
    resid_conn2 = noramlised_resid_conn+ff_network
    normalised_resid_conn = self.layer_norm4(resid_conn2)
    return normalised_resid_conn


class MyDecoder(nn.Module):
  def __init__(self, vocab_size, max_seq_len, d_model, num_heads, ff_hidden, num_decoder_layers):
    super().__init__()
    self.embedding_layer = MyEmbedding(d_model, vocab_size)
    self.positional_encoding = MyPositionalEncoding(max_seq_len, d_model)
    self.decoder_layers = nn.ModuleList([MyDecoderLayer(d_model, num_heads, ff_hidden) for _ in range(num_decoder_layers)])
    self.output_layer = MyOutputLayer(d_model, vocab_size)

  def forward(self, input, encoder_output, padding_mask):
    input = input.long()
    embed_input = self.embedding_layer(input)
    input = embed_input + self.positional_encoding(input)
    for layer in self.decoder_layers:
      input = layer(input, encoder_output, padding_mask)
    logits = self.output_layer(input)
    return logits

class MyTransformer(nn.Module):
  def __init__(self, vocab_size, max_seq_len, d_model, num_heads, ff_hidden, num_encoder_layers, num_decoder_layers):
    super().__init__()
    self.encoder = MyEncoder(vocab_size, max_seq_len, d_model, num_heads, ff_hidden, num_encoder_layers)
    self.decoder = MyDecoder(vocab_size, max_seq_len, d_model, num_heads, ff_hidden, num_decoder_layers)

  def forward(self, source, target, source_padding_mask, target_padding_mask):
    encoder_output = self.encoder(source, source_padding_mask)
    decoder_output = self.decoder(target, encoder_output, target_padding_mask) #raw logits, F.CrossEntropy already does softmax

    return decoder_output

In [3]:
def loss_function(target, model_output):
  # target is (batch_size, seq_len), no embedding, essnetially  label per sample (sample being one of seq_len*batch), ie all positions
  # model output is (batch_size, seq_len, vocab_size)
  vocab_size = model_output.size(-1)
  return F.cross_entropy(model_output.reshape(-1, vocab_size), target.reshape(-1))


def generate(model, max_seq_len):
  # inference from ready trained model with greedy appraoch
  return

# Training on Huggingface data

## Laoding in WMT16 from Huggingface interface

In [4]:
import torch
torch.cuda.is_available()

True

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [ ]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
dataset = load_dataset('wmt16','de-en', split="train[:1%]")


In [ ]:
def preprocess_function(dataset_sample, tokenizer=tokenizer):
  source_seq = tokenizer(dataset_sample["translation.de"], max_length=128, truncation=True, padding="max_length")
  target_seq = tokenizer(dataset_sample["translation.en"], max_length=128, truncation=True, padding="max_length")
  source_seq["labels"] = target_seq
  return source_seq
dataset = dataset.flatten()
preprocessed_dataset = dataset.map(preprocess_function, batch_size=1000, num_proc=4, remove_columns=["translation.de", "translation.en"])

In [ ]:
max_size = 0
for batch in train_dataloader:
  source_inpt = batch["input_ids"].long()
  max_size = max(max_size, source_inpt.size(1))
max_size

In [ ]:
preprocessed_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
train_dataloader = DataLoader(preprocessed_dataset, batch_size=10, shuffle=True)

In [ ]:
! export CUDA_LAUNCH_BLOCKING=1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from torch.optim import Adam
from torch.utils.data import DataLoader

config_params = {
    "d_model": [20],
    "num_heads": [4],
    "ff_hidden": [10],
    "num_encoder_layers" : [2],
    "num_decoder_layers": [2],
    "learning_rate": [1e-4]
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = tokenizer.vocab_size
max_seq_len = 200 # as set in the preprocess fct
num_epochs = 10

for d_model in config_params["d_model"]:
  for num_heads in config_params["num_heads"]:
    for ff_hidden in config_params["ff_hidden"]:
      for num_encoder_layers in config_params["num_encoder_layers"]:
        for num_decoder_layers in config_params["num_decoder_layers"]:
          for lr in config_params["learning_rate"]:
            transformer_model = MyTransformer(vocab_size, max_seq_len, d_model, num_heads, ff_hidden, num_encoder_layers, num_decoder_layers)
            transformer_model = transformer_model.to(device)
            optim = Adam(transformer_model.parameters(), lr=lr)

            for epoch in range(num_epochs):
              transformer_model.train()
              total_loss = 0

              for batch in train_dataloader:

                source_inpt = batch["input_ids"].long()
                source_mask = batch["attention_mask"].long()
                target = batch["labels"]["input_ids"].long()
                loss_fct_target = target[:, 1:] #targets for loss function start after first token
                decoder_input = target[:, :-1]


                target = target.to(device)

                target_mask = batch["labels"]["attention_mask"]
                target_mask = target_mask.to(device)
                target_mask = target_mask[:, :-1] #during training the model should not see the next token
                source_inpt = source_inpt.to(device)
                source_mask = source_mask.to(device)
                decoder_input = decoder_input.to(device)

                optim.zero_grad() # reset the gradients
                logits = transformer_model(source_inpt, decoder_input, source_mask, target_mask)
                loss = loss_function(loss_fct_target, logits)
                loss.backward()
                optim.step()
                total_loss += loss.item()
              if epoch % 2 ==0:
                print(total_loss//dataset.size(0)) # loss per sample in batch

In [ ]:
test = MyTransformer(vocab_size, max_seq_len, d_model, num_heads, ff_hidden, num_encoder_layers, num_decoder_layers)


In [ ]:
test.to(device)

In [ ]:
for batch in train_dataloader:
  source_inpt = batch["input_ids"]
  target = batch["labels"]["input_ids"]
  loss_fct_target = target[:, 1:] #targets for loss function start after first token
  decoder_input = target[:, :-1]
  print(source_inpt.size())
  print(decoder_input.size())
  break